In [1]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import math
tf.disable_v2_behavior()

con = sqlite3.connect('../summer')
cur = con.cursor()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']
recordings = ['\'week1_3\'', '\'week2_3\'', '\'week3_3\'', '\'week4_3\'', '\'week5_3\'', '\'week6_3\'']

all_weeks = pd.read_sql_query("""SELECT PlayerName, RecordingName,
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings)), con)

print(all_weeks.shape)

(3189974, 23)


In [5]:
translation_columns = ['HeadPos_x', 'HeadPos_y', 'HeadPos_z',
                       'LeftHandPos_x', 'LeftHandPos_y', 'LeftHandPos_z',
                       'RightHandPos_x', 'RightHandPos_y', 'RightHandPos_z']
rotation_columns = ['HeadRot_w', 'HeadRot_x', 'HeadRot_y', 'HeadRot_z',
                    'LeftHandRot_w', 'LeftHandRot_x', 'LeftHandRot_y', 'LeftHandRot_z',
                    'RightHandRot_w', 'RightHandRot_x', 'RightHandRot_y', 'RightHandRot_z']

playernames = all_weeks['PlayerName'].unique()
recordingnames = all_weeks['RecordingName'].unique()

playerIdx = {
    'Alonzo': 0,
    'Dax': 1,
    'Emily': 2,
    'Isabelle': 3,
    'Jake': 4,
    'Philip': 5,
    'Rishab': 6,
    'Sam': 7,
    'justin': 8,
    'Alex': 9,
    'Jordan': 10
}

motion_input = []
motion_output = []

for recording_name in recordingnames:
    recording_df = all_weeks[all_weeks['RecordingName'] == recording_name]

    for name in playernames:
        player_df = recording_df[recording_df['PlayerName'] == name]

        last_frame = 0
        for i in range(0, player_df.shape[0] - 59, 30):
            features = []
            window = player_df.iloc[i:i+59]

            for column_name in translation_columns:
                column = window[column_name]
                features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

                avg_velocity = (column.iloc[-1] - column.iloc[0]) / 2.0
                avg_speed = column.abs().sum() / 2.0

                features.extend([avg_velocity, avg_speed])

            for column_name in rotation_columns:
                column = window[column_name]
                features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

            motion_input.append(features)
            output = np.zeros(len(playernames))
            output[playerIdx[name]] = 1
            motion_output.append(output)
            last_frame = i

        last_window = player_df.iloc[last_frame + 30: -1]
        final_features = []
        for column_name in translation_columns:
            column = window[column_name]
            final_features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

            avg_velocity = (column.iloc[-1] - column.iloc[0]) / 2.0
            avg_speed = column.abs().sum() / 2.0

            final_features.extend([avg_velocity, avg_speed])

        for column_name in rotation_columns:
            column = window[column_name]
            final_features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

        motion_input.append(final_features)
        output = np.zeros(len(playernames))
        output[playerIdx[name]] = 1
        motion_output.append(output)

print(len(motion_input))
print(len(motion_input[0]))

print(len(motion_output))
print(len(motion_output[0]))

106309
144
106309
11


In [6]:
training_x = np.asarray(motion_input)
training_y = np.asarray(motion_output)

In [7]:
recordings_test = ['\'week7_3\'', '\'week8_3\'']

all_weeks_test = pd.read_sql_query("""SELECT PlayerName, RecordingName,
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings_test)), con)

print(all_weeks_test.shape)

(919446, 23)


In [8]:
playernames_w2 = all_weeks_test['PlayerName'].unique()
recordingnames_w2 = all_weeks_test['RecordingName'].unique()

motion_input_w2 = []
motion_output_w2 = []

for recording_name in recordingnames_w2:
    recording_df = all_weeks_test[all_weeks_test['RecordingName'] == recording_name]

    for name in playernames_w2:
        player_df = recording_df[recording_df['PlayerName'] == name]

        last_frame = 0
        for i in range(0, player_df.shape[0] - 59, 30):
            features = []
            window = player_df.iloc[i:i+59]

            for column_name in translation_columns:
                column = window[column_name]
                features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

                avg_velocity = (column.iloc[-1] - column.iloc[0]) / 2.0
                avg_speed = column.abs().sum() / 2.0

                features.extend([avg_velocity, avg_speed])

            for column_name in rotation_columns:
                column = window[column_name]
                features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

            motion_input_w2.append(features)
            output = np.zeros(len(playernames))
            output[playerIdx[name]] = 1
            motion_output_w2.append(output)
            last_frame = i

        last_window = player_df.iloc[last_frame + 30: -1]
        final_features = []
        for column_name in translation_columns:
            column = window[column_name]
            final_features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

            avg_velocity = (column.iloc[-1] - column.iloc[0]) / 2.0
            avg_speed = column.abs().sum() / 2.0

            final_features.extend([avg_velocity, avg_speed])

        for column_name in rotation_columns:
            column = window[column_name]
            final_features.extend([column.min(), column.max(), column.mean(), column.std(), column.mad(), column.sum()])

        motion_input_w2.append(final_features)
        output = np.zeros(len(playernames))
        output[playerIdx[name]] = 1
        motion_output_w2.append(output)

print(len(motion_input_w2))
print(len(motion_input_w2[0]))

print(len(motion_output_w2))
print(len(motion_output_w2[0]))

30639
144
30639
11


In [9]:
testing_x = np.asarray(motion_input_w2)
testing_y = np.asarray(motion_output_w2)

## Neural Network Training

In [10]:
class Data:

    def __init__(self, training_X, training_y, testing_X, testing_y, num_examples, input_dim, output_dim):
        self.training_X = training_X
        self.training_y = training_y
        self.testing_X = testing_X
        self.testing_y = testing_y
        self.num_examples = num_examples
        self.index_in_epoch = 0
        self.epochs_completed = 0
        self.input_dim = input_dim
        self.output_dim = output_dim

    def next_batch(self, batch_size):
        start = self.index_in_epoch
        self.index_in_epoch += batch_size
        if self.index_in_epoch > self.num_examples:
            self.epochs_completed += 1
            perm = np.arange(self.num_examples)
            np.random.shuffle(perm)
            self.training_X = self.training_X[perm]
            self.training_y = self.training_y[perm]
            start = 0
            self.index_in_epoch = batch_size
            assert batch_size <= self.num_examples
        end = self.index_in_epoch
        return self.training_X[start:end], self.training_y[start:end]

In [11]:
class MotionData(Data):

    def __init__(self):
        Data.__init__(self, training_x, training_y, testing_x, testing_y, 106309, 144, 11)

data = MotionData()

In [12]:
import time
import os

class ClassificationNN:
    def __init__(self,dataset,learning_rate,training_iteration,batch_size, hidden_layer_n):
        self.dataset = data

        self.learning_rate = learning_rate
        self.training_iteration = training_iteration
        self.batch_size = batch_size
        input_layer_n = self.dataset.input_dim
        output_layer_n = self.dataset.output_dim
        self.display_step = 3
        
        tf.reset_default_graph()

        self.x = tf.placeholder("float", [None, input_layer_n])
        self.y = tf.placeholder("float", [None, output_layer_n])

        W1 = tf.Variable(tf.random_normal([input_layer_n, hidden_layer_n], stddev=0.1), name='W1')
        W2 = tf.Variable(tf.random_normal([hidden_layer_n, output_layer_n], stddev=0.1), name='W2')

        b1 = tf.Variable(tf.random_normal([hidden_layer_n], stddev=0.1), name='b1')
        b2 = tf.Variable(tf.random_normal([output_layer_n], stddev=0.1), name='b2')

        layer_1 = tf.nn.sigmoid(tf.matmul(self.x, W1) + b1)
        layer_2 = tf.nn.softmax(tf.matmul(layer_1, W2) + b2)
        self.model = layer_2

        self.objective_function = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.model, self.y))))

        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.objective_function)

        self.init = tf.global_variables_initializer()
        
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            print('start training: ', time.strftime("%H:%M:%S", time.localtime()))
            sess.run(self.init)
            for iteration in range(self.training_iteration):
                avg_cost = 0
                num_batches = int(self.dataset.num_examples/self.batch_size)
                for i in range(num_batches):
                    batch_xs, batch_ys = self.dataset.next_batch(self.batch_size)
                    sess.run(self.optimizer, feed_dict={self.x: batch_xs, self.y: batch_ys})
                    avg_cost += sess.run(self.objective_function, feed_dict={self.x: batch_xs, self.y: batch_ys})/num_batches
                if iteration % self.display_step == 0:
                    print("Epoch:" + str(iteration+1) +  " cost: ", "{:.9f}".format(avg_cost))

            print("Training completed!", time.strftime("%H:%M:%S", time.localtime()))
  
            predictions = tf.equal(tf.argmax(self.model, 1), tf.argmax(self.y, 1))
            accuracy = tf.reduce_mean(tf.cast(predictions,  tf.float64))
            print("Testing accuracy:", accuracy.eval({self.x: self.dataset.testing_X, self.y: self.dataset.testing_y}))
            
            #end_time = str(time.strftime("%d_%H_%M_%S", time.localtime()))
            #os.makedirs('models_FE/' + end_time)
            
            #np.savetxt('models_FE/' + end_time + 'W1.csv', W1.eval())
            #np.savetxt('models_FE/' + end_time + 'W2.csv', W2.eval())
            #np.savetxt('models_FE/' + end_time + 'b1.csv', b1.eval())
            #np.savetxt('models_FE/' + end_time + 'b2.csv', b2.eval())
            
            #print(saver._var_list)
            #saver.save(sess, 'models_FE/' + end_time + '/FE' + end_time)

In [13]:
nn = ClassificationNN(data, learning_rate=0.0005,batch_size=50,training_iteration=100, hidden_layer_n=200)

2022-04-20 08:50:13.052876: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


start training:  08:50:13
Epoch:1 cost:  6.227541351
Epoch:4 cost:  6.428728702
Epoch:7 cost:  6.341290302
Epoch:10 cost:  6.268417973
Epoch:13 cost:  6.191671085
Epoch:16 cost:  6.134830729
Epoch:19 cost:  6.084256710
Epoch:22 cost:  6.037662985
Epoch:25 cost:  6.025204507
Epoch:28 cost:  5.971876772
Epoch:31 cost:  5.947691776
Epoch:34 cost:  5.907905389
Epoch:37 cost:  5.923364282
Epoch:40 cost:  5.873566554
Epoch:43 cost:  5.885456736
Epoch:46 cost:  5.859959399
Epoch:49 cost:  5.829033619
Epoch:52 cost:  5.836688677
Epoch:55 cost:  5.880300491
Epoch:58 cost:  5.919304980
Epoch:61 cost:  5.876915048
Epoch:64 cost:  5.822350851
Epoch:67 cost:  5.838757869
Epoch:70 cost:  5.845977406
Epoch:73 cost:  5.807181917
Epoch:76 cost:  5.779558613
Epoch:79 cost:  5.782067915
Epoch:82 cost:  5.810580770
Epoch:85 cost:  5.836528021
Epoch:88 cost:  5.794310851
Epoch:91 cost:  5.779914532
Epoch:94 cost:  5.802515332
Epoch:97 cost:  5.793122221
Epoch:100 cost:  5.764344636
Training completed! 08:5